## **Exploration purpose: using Andrej Karpathy base code**

##**Base Kaparthy tutorial**

In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time
# data I/O
path_to_file = '/content/shakespeare_train.txt'
data = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

Streaming output truncated to the last 5000 lines.

Bot:
Busbho hlactong I will rveeny cecofenginsey we h 
----
iter 53100, loss: 50.955298
----
 ner ming niuse prounp,
Whir glacuin?

LARIUS:
Htom spain you afespanic.

GUCIVIY GERHAM:
Yo bean,
Thiakmyar: o inond:
And atdel is alathy mathinss mes.
And tobe thoWh to your tho toon, theerd that sma 
----
iter 53200, loss: 50.997145
----
 ke gon thew, hade, dom recent, sI tery ghat at and nokle and my tisotid warer yot bast ot mosersee Carficilknge: we that his engaff soglyien a knay tashar now toblith
wertwoulce!
Your o nisisen
The th 
----
iter 53300, loss: 51.036311
----
 , nes megrs my cawingey therrew lin rave.

Sefill: than his kindearsbup ae, dectall wakeny all core his that iite, to to declore warcond is. Ghe sous,
Wo didshel theep kare.
Thely'd thas afmene!
Nou e 
----
iter 53400, loss: 50.945399
----
 ld sineigh hibt at
leniloot:
I hy way, may heak, m.

CORGINETINGS:
Code ciss en so to wive;
Te. Ridsed of hestled, came cotws.

Sec

KeyboardInterrupt: ignored

## **Same with loaded weights on 1000 iterations**

In [ ]:
import numpy as np

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
data = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#data = open('shakespeare_train.txt', 'r').read() # should be simple plain text file

# Using the trained weights 
with np.load('/content/char-rnn-snapshot.npz', allow_pickle=True) as a:

    Wxh = a["Wxh"] # 250 x 62
    Whh = a["Whh"] # 250 x 250
    Why = a["Why"] # 62 x 250
    bh = a["bh"] # 250 x 1
    by = a["by"] # 62 x 1
    chars, data_size, vocab_size, char_to_ix, ix_to_char = a["chars"].tolist(), a["data_size"].tolist(), a["vocab_size"].tolist(), a["char_to_ix"].tolist(), a["ix_to_char"].tolist()

# hyperparameters
hidden_size = 250
seq_length = 1000 # number of steps to unroll the RNN for

# Part 1, Generating Samples
def temp(length, alpha=1):
  """
  generate a sample text with assigned alpha value for temperture.
  """
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  inputs = [char_to_ix[ch] for ch in data[:seq_length]]
  hs = np.zeros((hidden_size,1))
  
  # generates a sample
  sample_ix = sample(hs, inputs[0], length, alpha)
  txt = ''.join(ix_to_char[ix] for ix in sample_ix)
  print ('----\n%s \n----' % (txt, ))                            

  
def sample(h, seed_ix, n, alpha):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(alpha * y) / np.sum(np.exp(alpha * y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes
  
  
#Complete stringdef comp(m, n):
  """
  given a string with length m, complete the string with length n more characters
  """
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  np.random.seed
  # the context string starts from a random postiion in the data
  start_index = np.random.randint(265000) 
  inputs = [char_to_ix[ch] for ch in data[start_index : start_index+seq_length]]
  h = np.zeros((hidden_size,1))
  x = np.zeros((vocab_size, 1))
  word_index = 0
  ix = inputs[word_index]
  x[ix] = 1

  ixes = []
  ixes.append(ix)
  for t in range(m):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    # x is one of k encoding of which index is 1 in the char
    x = np.zeros((vocab_size, 1))
    ix = inputs[word_index + 1]
    word_index += 1
    x[ix] = 1
    ixes.append(ix)
    
  # generates the context text
  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('Context: \n----\n%s \n----\n\n\n' % (txt, ))
  
  # compute the softmax probability and sample from the data
  # and use the output as the next input where we start the continuation
  y = np.dot(Why, h) + by
  p = np.exp(y) / np.sum(np.exp(y))
  ix = np.random.choice(range(vocab_size), p=p.ravel())
  x = np.zeros((vocab_size, 1))
  x[ix] = 1

  # start completing the string
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  
  # generates the continuation of the string
  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('Continuation: \n----\n%s \n----' % (txt, ))
  

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
temp(1000,alpha=0.9)

----
irst Sess,
Ant eave?

SICINIUS:
It
And trugh, cheme, nobam,
melpoubllee ligen; crre wise tril strend lath fasmed Vimp
A weles,
Stelurk's foulf:
And kituum feries I the them lish cargent ongl:, a mise streing they huse cad neverant Molfiks did had this for my do what prove thee you.

CORs.

AERIA:
The pariple dake o' the pay they astuen
A meres that Lord hims I figh nave che for one fay, when; my bave's, naind it themil whlaus: havp to heam, noisfaant; higheling: see no doftarfe hot noth?

SICGNAEORINIUS:
Gety How good blere, quor god, Comlae. but she senee hart, hrougnfill full no nuthernsse come.

SICINIUS:
You mo I hive good te Who lirlm, ot.

BRUTUS:
Held had fon trell'er nepaue make your.
Mogl-bould,
He's husere mlsalact logh!
What,
Minow: o'ttes! ELE EDhIVat porsmether
Home in Shaild,

MESS
IUSUS:
That, the it of nats coun'.

FyOUCESTOUSHOUS:
If re, the corlvecy!
in? I deak farerrer.
O, I woung hoss whon, gul Vomes Cay, and WCowand lord, yet mat.

BRUTUS:

MARCIUSAIUS:
Horuce

In [ ]:
comp(300,300)

Context: 
----
ehouse gratis, as 'twas used
Sometime in Greece,--

MENENIUS:
Well, well, no more of that.

CORIOLANUS:
Though there the people had more absolute power,
I say, they nourish'd disobedience, fed
The ruin of the state.

BRUTUS:
Why, shall the people give
One that speaks thus their voice?

CORIOLANUS:
I' 
----



Continuation: 
----
wo; now.

First Senath haves,--

BRUTUS:
And for or dires omion thougy I take clactrk!
I'le.
Bo ta's igas you wound ho the amke.

BRUTUS:
Got ager:
Thatsae rake See had thes! reneshery; shus losd the cads,
Than de canode?

MENENIUS:
And can afe feast
The stand all are he.

CORIOLANUS:
No, stCing,
Mo 
----


# Markov model (char/word level)

In [ ]:
import io
path ='/content/shakespeare_train.txt'
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()


##Dict/lookup list

In [ ]:
from collections import defaultdict

def markov_chain(text):
    
    words = text.split(' ')
    
    m_dict = defaultdict(list)
    
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    m_dict = dict(m_dict)
    return m_dict

In [ ]:
_dictwords = markov_chain(text)


In [ ]:
_dictwords

{'first': ['citizen:\nbefore',
  'to',
  "unroof'd",
  'hearing',
  'seeing',
  'budger',
  'complaint;',
  'we',
  "way.\n\nmenenius:\ni'll",
  'a',
  'son.\nwhither',
  'my',
  'to',
  'i',
  'he',
  'conditions,',
  'time',
  "i'll",
  'begin',
  'that',
  "let's",
  'order',
  'he'],
 'citizen:\nbefore': ['we', 'the'],
 'we': ['proceed',
  "know't.\n\nfirst",
  'might',
  'are',
  'become',
  'prating',
  'intend',
  'are',
  'shall',
  'were',
  'proceed.\n\naufidius:\nis',
  'were',
  'and',
  'shall',
  'hear',
  'with',
  'have',
  'have',
  'wish',
  'heard',
  'have',
  'have',
  'lose',
  'the',
  'have',
  'do',
  'will',
  'may',
  'no',
  'to',
  'encounter\nas',
  'eat.',
  'shall',
  'censured?\n\nmenenius:\nbecause',
  'know',
  'stand,',
  'to',
  "call'd\nboth",
  'devise',
  'all',
  'ought',
  'will.\n\nthird',
  'have',
  'are',
  'must',
  'being',
  'stood',
  'are',
  'give',
  'hope',
  'do',
  "do't,\nthe",
  'granted',
  'may',
  'laboured,\nno',
  'read',
 

##generate

In [ ]:
import random

def generate_sentence(chain, count=100):


    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()
    #sentence="So will you come"
    print("Seed sentence: ",sentence)
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    print(sentence)

In [ ]:
generate_sentence(_dictwords)

Seed sentence:  So will you come
So will you come not do,
i'll drown the humane way: the next room.

second murderer:
o excellent devise! make road.
upon's again.

cominius:
they are the welcome you.

york:
what, will pluck out
the multitudinous tongue; let the noble vessel: what's not flatter, and
therein behold now
presently, and eat him i shall bear
a better than the chiefest soldiers; i'll bear against yourself.

queen margaret:
poor painted queen, vain flourish of warriors,
how is dead?
you do it, and i inform them.

brutus:
and when you know, it not whether he is the superfluity, while the boar did inform them!

coriolanus:
you are stronger earth he the roman and so doing, our suit of the rich, and that: if thou whet'st.


# Char level

# LSTM & GRU

In [ ]:
from __future__ import absolute_import, division,print_function, unicode_literals

import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM,GRU

from tensorflow.keras.optimizers import RMSprop,Adam

from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import random
import sys


In [ ]:

with open('/content/shakespeare_train.txt', 'r') as file:
	text = file.read()



##preprocess, dict

In [ ]:
vocabulary = sorted(list(set(text)))

char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))
print(char_to_indices)

print(indices_to_char)


{'\n': 0, ' ': 1, '!': 2, '&': 3, "'": 4, ',': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 'N': 24, 'O': 25, 'P': 26, 'Q': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'Y': 34, 'Z': 35, 'a': 36, 'b': 37, 'c': 38, 'd': 39, 'e': 40, 'f': 41, 'g': 42, 'h': 43, 'i': 44, 'j': 45, 'k': 46, 'l': 47, 'm': 48, 'n': 49, 'o': 50, 'p': 51, 'q': 52, 'r': 53, 's': 54, 't': 55, 'u': 56, 'v': 57, 'w': 58, 'x': 59, 'y': 60, 'z': 61}
{0: '\n', 1: ' ', 2: '!', 3: '&', 4: "'", 5: ',', 6: '-', 7: '.', 8: ':', 9: ';', 10: '?', 11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F', 17: 'G', 18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L', 23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R', 29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'Y', 35: 'Z', 36: 'a', 37: 'b', 38: 'c', 39: 'd', 40: 'e', 41: 'f', 42: 'g', 43: 'h', 44: 'i', 45: 'j', 46: 'k', 47: 'l', 48: 'm', 49: 'n', 50: 'o',

In [ ]:

max_length = 100
steps = 5
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, steps):
	sentences.append(text[i: i + max_length])
	next_chars.append(text[i + max_length])

X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool)
	
# Placing the value 1 at the appropriate position for each vector
# to complete the hot-encoding process
for i, sentence in enumerate(sentences):
	for t, char in enumerate(sentence):
		X[i, t, char_to_indices[char]] = 1
	y[i, char_to_indices[next_chars[i]]] = 1
print(X[1][1])
print(y[1])


[False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False]


##GRU model

In [ ]:
#used stacked GRU
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential(
    [
        
        layers.GRU(256, input_shape =(max_length, len(vocabulary)), return_sequences=True),
        layers.Dropout(0.2),
        layers.GRU(128),
        layers.Dropout(0.2),
        layers.Dense(len(vocabulary), activation="softmax"),
    ]
)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
'''

# Initializing the LSTM network
model = Sequential()

# Defining the cell type
model.add(GRU(128, input_shape =(max_length, len(vocabulary))))

# Defining the densely connected Neural Network layer
model.add(Dense(len(vocabulary)))

# Defining the activation function for the cell
model.add(Activation('softmax'))

# Defining the optimizing function
optimizer = RMSprop(lr = 0.01)

# Configuring the model for training
model.compile(loss ='categorical_crossentropy', optimizer = optimizer)
'''


"\n\n# Initializing the LSTM network\nmodel = Sequential()\n\n# Defining the cell type\nmodel.add(GRU(128, input_shape =(max_length, len(vocabulary))))\n\n# Defining the densely connected Neural Network layer\nmodel.add(Dense(len(vocabulary)))\n\n# Defining the activation function for the cell\nmodel.add(Activation('softmax'))\n\n# Defining the optimizing function\noptimizer = RMSprop(lr = 0.01)\n\n# Configuring the model for training\nmodel.compile(loss ='categorical_crossentropy', optimizer = optimizer)\n"

##Generation function

In [ ]:
def sample_index(preds, temperature = 1.0):
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)


In [ ]:
def on_epoch_end(epoch, logs):
	print()
	print('----- Generating text after Epoch: % d' % epoch)
	start_index = random.randint(0, len(text) - max_length - 1)

	for diversity in [0.2, 0.5, 1.0, 1.2]:
		print('----- diversity:', diversity)

		generated = ''

		sentence = text[start_index: start_index + max_length]

		generated += sentence
		print('----- Generating with seed: "' + sentence + '"')
		sys.stdout.write(generated)

		for i in range(400):
			x_pred = np.zeros((1, max_length, len(vocabulary)))

			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			preds = model.predict(x_pred, verbose = 0)[0]

			next_index = sample_index(preds, diversity)

			next_char = indices_to_char[next_index]

			generated += next_char
			sentence = sentence[1:] + next_char

			sys.stdout.write(next_char)
			sys.stdout.flush()
		print()

print_callback = LambdaCallback(on_epoch_end = on_epoch_end)


In [ ]:

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor ='loss',
							verbose = 1, save_best_only = True,
							mode ='min')


In [ ]:

reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2,
							patience = 1, min_lr = 0.001)
callbacks = [print_callback, checkpoint, reduce_alpha]


In [ ]:
model.fit(X, y, batch_size = 128, epochs = 30, callbacks = callbacks)


Epoch 1/30
420/420 [==============================] - 11s 21ms/step - loss: 2.3467

----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "STER:
Say that I slew them not?

LADY ANNE:
Why, then they are not dead:
But dead they are, and devi"
STER:
Say that I slew them not?

LADY ANNE:
Why, then they are not dead:
But dead they are, and devines the wire and in ener is and in be to the mene be to be and is in the world is as is is be to merer is and is and leide and in be till the would will would will he ware and in merene be to the world and is enene a and is is till the world will beece and a and in is is be bened and in till he world me to merer and ine all is the wolld is seed is lied is the world he would be and in is is in be a
----- diversity: 0.5
----- Generating with seed: "STER:
Say that I slew them not?

LADY ANNE:
Why, then they are not dead:
But dead they are, and devi"
STER:
Say that I slew them not?

LADY ANNE:
Why, then they are not dead:


In [ ]:
#stacked GRU output

def generate_text(length, diversity):
	start_index = random.randint(0, len(text) - max_length - 1)

	generated = ''
	sentence = text[start_index: start_index + max_length]
	generated += sentence

	for i in range(length):

			x_pred = np.zeros((1, max_length, len(vocabulary)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			preds = model.predict(x_pred, verbose = 0)[0]

			next_index = sample_index(preds, diversity)

			next_char = indices_to_char[next_index]

			#  new text
			generated += next_char
			sentence = sentence[1:] + next_char
	return generated

print(generate_text(500, 0.2))


 Caius, and
By deed-achieving honour newly named,--
What is it?--Coriolanus must I call thee?--
But Of I cans the king of your grace of some his country;
And but not stand you have be so mother, you all the people

AUFIDIUS:
If I have as the grace and he word show for be the king of your grace?

GLOUCESTER:
I would not shall seem the bear so the bring the all and more and her for it.

GLOUCESTER:
I would the noble for the vilied thy son, when you.

VOLUMNIA:
I have and the grain.

BRUTUS:
He's a shall the brows be be not a botier.

First Senator:
Now, be there shall be them as be so be them to


##GRU fit

In [ ]:
##single layer
model.fit(X, y, batch_size = 128, epochs = 30, callbacks = callbacks)


Epoch 1/30
420/420 [==============================] - 15s 21ms/step - loss: 2.2261

----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "g. Stand, Aufidius,
And trouble not the peace.

CORIOLANUS:
O that I had him,
With six Aufidiuses, o"
g. Stand, Aufidius,
And trouble not the peace.

CORIOLANUS:
O that I had him,
With six Aufidiuses, of there have there of thered inder to thered of thered so deard so so diver:
Wher my there to heresed so sould so dite to there sond so so to dieder in to ster to hised to sterd inder to deed
There sondered so shard there there of ther my sorded to have so diver dise.

First Seaven:
For there to stered to there of there hised to thered so divered sondser:
There so so there stord so have shorded so
----- diversity: 0.5
----- Generating with seed: "g. Stand, Aufidius,
And trouble not the peace.

CORIOLANUS:
O that I had him,
With six Aufidiuses, o"
g. Stand, Aufidius,
And trouble not the peace.

CORIOLANUS:
O that I had 

In [ ]:
def generate_text(length, diversity):
	start_index = random.randint(0, len(text) - max_length - 1)

	generated = ''
	sentence = text[start_index: start_index + max_length]
	generated += sentence

	for i in range(length):

			x_pred = np.zeros((1, max_length, len(vocabulary)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_to_indices[char]] = 1.

			preds = model.predict(x_pred, verbose = 0)[0]

			next_index = sample_index(preds, diversity)

			next_char = indices_to_char[next_index]

			#  new text
			generated += next_char
			sentence = sentence[1:] + next_char
	return generated

print(generate_text(500, 0.2))


brother's son:
So say we too, but not by Edward's wife;
For first he was contract to Lady Lucy--
Your grace to see me to a baster for the command
With all the steal the deeppich of the commoner loads
That it so us the strietce, the cornows, the like the comboul sured,
And the briefent with a partle to the storther did the deep
oppery and the grace of hold mine.

MENENIUS:
O, you all the like the man ever.

CORIOLANUS:
O, the like him the marrethirs, and the ressment
With the like the common much the people, were you will
And should a to the steal the storn and the displeed
The greater to be th


##LSTM

In [ ]:
model1 = Sequential()

model1.add(LSTM(128, input_shape =(max_length, len(vocabulary))))

model1.add(Dense(len(vocabulary)))

model1.add(Activation('softmax'))

optimizer = Adam(lr = 0.01)

model1.compile(loss ='categorical_crossentropy', optimizer = optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:

filepath1 = "weights1.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor ='loss',
							verbose = 1, save_best_only = True,
							mode ='min')


In [ ]:

reduce_alpha1 = ReduceLROnPlateau(monitor ='loss', factor = 0.2,
							patience = 1, min_lr = 0.001)
callbacks1 = [print_callback, checkpoint1, reduce_alpha1]


In [ ]:

model1.fit(X, y, batch_size = 128, epochs = 30, callbacks = callbacks1)


Epoch 1/30
420/420 [==============================] - 12s 24ms/step - loss: 2.5337

----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "is answer to me was,
He could not stay to pick them in a pile
Of noisome musty chaff: he said 'twas "
is answer to me was,
He could not stay to pick them in a pile
Of noisome musty chaff: he said 'twas dram
A high made me and the day as resome.

Second Servingman:
What's the matter father and the day so requires.

MENENIUS:
Why, he shall, some most noble ride,
And the stiel to the herght count it stand at he
cain with we have sen the still to the hole voices:
Who say, the cornow, and the like the people,
And the should be deed a poison, and the country's soul agains
And the contrain that dischan
----- diversity: 0.5
----- Generating with seed: "is answer to me was,
He could not stay to pick them in a pile
Of noisome musty chaff: he said 'twas "
is answer to me was,
He could not stay to pick them in a pile
Of noisome 

# Stacked GRU model worked better for the Shakespeare_train.tct

# Word level

##**LSTM**

In [ ]:
from __future__ import print_function
from keras.layers import Embedding,Bidirectional
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file

import numpy as np
import random
import sys
import os

path = "/content/shakespeare_train.txt"


text = open(path).read().lower()

text = open(path, encoding='utf-8').read().lower()

print('corpus length:', len(text))

chars = set(text)
text = text.replace(':', ':\n ')
words = set(text.lower().split())
print(words)
print("chars:",type(chars))
print("words",type(words))
print("total number of unique words",len(words))
print("total number of unique chars", len(chars))


word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

print("word_indices", type(word_indices), "length:",len(word_indices) )
print("indices_words", type(indices_word), "length", len(indices_word))

maxlen = 30
step = 3
print("maxlen:",maxlen,"step:", step)
sentences = []
next_words = []
next_words= []
sentences1 = []
list_words = []

sentences2=[]
list_words=text.lower().split()


for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
print('length of sentences:', len(sentences))
print("length of next_word",len(next_words))

#vectors
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1
seq_length = X.shape[1]
vocab_size=len(words)


corpus length: 268330
{'utter,', 'wrong.', 'voices!', 'gently', 'revolt;', 'twist', 'shapes,', 'colic,', 'beauty', 'sir,--', 'substitute,', 'strong', 'guest.', 'stayed', 'slightly', 'sometime', 'voices:', 'melancholy', 'quiet;', 'buttock', 'infirmity.', 'gloucester;', 'hearts!', 'grained', 'names;', 'reports,', 'ended', 'violent,', 'sulphur', 'gone:', "clapp'd", 'lodged', 'to,', 'valiant.', 'composition.', 'canon.', 'virtue,', "resign'd", 'success,', 'hopeful', 'ought', 'replied,', 'breath', 'judgment?', 'forgot.', 'suddenly', 'deucalion,', 'christendom', 'measles,', 'past.', 'incur', 'boys:', "o'er-peer.", 'not?', "smear'd;", 'provokes', 'between,', 'comely', 'killing', 'mutinous;', 'bishops.', 'hidden', 'heart.', 'storehouse', 'dinner:', 'prophecies', 'not--to', 'forced', 'souls.', "prince!'", 'glasses', 'many', 'worthier', 'lancaster!', 'showing', 'lawful', "'zounds!", 'children?', 'pamper', 'expel', 'a', "upon't!", 'protest,', 'doit,', 'good,', 'lartius', "rome's", "dogg'd", "anoth

In [ ]:
X[1][1]

array([False, False, False, ..., False, False, False])

In [ ]:
y[1]

array([False, False, False, ..., False, False, False])

##plain word level lstm

In [ ]:
'''
# define model
from keras.layers import Dense,LSTM,Embedding,Bidirectional
from keras.models import Sequential,Model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
'''
from tensorflow.keras.optimizers import Adam

#build the model: 2 stacked LSTM
model = Sequential()

model.add((LSTM(256, return_sequences=True, input_shape=(maxlen, len(words)))))

model.add((LSTM(128, return_sequences=True)))

model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))
optimizer=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

if os.path.isfile('GoTweights'):
    model.load_weights('GoTweights')



##BiLSTM

In [ ]:
'''
# define model
from keras.layers import Dense,LSTM,Embedding,Bidirectional
from keras.models import Sequential,Model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
'''
from tensorflow.keras.optimizers import Adam

#build the model: 2 stacked LSTM
model = Sequential()

model.add(Bidirectional(LSTM(256, return_sequences=True, input_shape=(maxlen, len(words)))))

model.add(Bidirectional(LSTM(128, return_sequences=True)))

model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))
optimizer=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

if os.path.isfile('GoTweights'):
    model.load_weights('GoTweights')



In [ ]:

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor ='loss',
							verbose = 1, save_best_only = True,
							mode ='min')


In [ ]:

reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2,
							patience = 1, min_lr = 0.001)
callbacks = [checkpoint, reduce_alpha]


##generate LSTM

In [ ]:
def sample(a, temperature=1.0):
    a = np.asarray(a).astype('float64')
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, epochs=2,callbacks=callbacks)
    model.save_weights('GoTweights',overwrite=True)

    start_index = random.randint(0, len(list_words) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = list_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)+'\n'
        print('----- Generating with seed: "' , sentence , '"')
        print('\n')
        sys.stdout.write(generated)
        print()

        for i in range(400):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()
#model.save_weights('weights') 


--------------------------------------------------
Iteration 1
Epoch 1/2
126/126 [==============================] - 109s 866ms/step - loss: 6.8847

Epoch 00001: loss improved from 6.92495 to 6.88471, saving model to weights.hdf5
Epoch 2/2
126/126 [==============================] - 110s 869ms/step - loss: 6.8510

Epoch 00002: loss improved from 6.88471 to 6.85103, saving model to weights.hdf5

----- diversity: 0.2
----- Generating with seed: " ['set', 'you', 'on', 'to', 'do', 'this', 'deed', 'will', 'hate', 'you', 'for', 'the', 'deed.', 'second', 'murderer:', 'what', 'shall', 'we', 'do?', 'clarence:', 'relent,', 'and', 'save', 'your', 'souls.', 'first', 'murderer:', 'relent!', "'tis", 'cowardly'] "


set you on to do this deed will hate you for the deed. second murderer: what shall we do? clarence: relent, and save your souls. first murderer: relent! 'tis cowardly

 and of the the of the the the the of the the of the the the and the the of the and the the i the and the the the the the 

##generate BiLSTM

In [ ]:
def sample(a, temperature=1.0):
    a = np.asarray(a).astype('float64')
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, epochs=2,callbacks=callbacks)
    model.save_weights('GoTweights',overwrite=True)

    start_index = random.randint(0, len(list_words) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = list_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)+'\n'
        print('----- Generating with seed: "' , sentence , '"')
        print('\n')
        sys.stdout.write(generated)
        print()

        for i in range(400):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()
#model.save_weights('weights') 


--------------------------------------------------
Iteration 1
Epoch 1/2
126/126 [==============================] - 61s 416ms/step - loss: 7.4000

Epoch 00001: loss improved from inf to 7.39999, saving model to weights.hdf5
Epoch 2/2
126/126 [==============================] - 53s 418ms/step - loss: 6.9442

Epoch 00002: loss improved from 7.39999 to 6.94420, saving model to weights.hdf5

----- diversity: 0.2
----- Generating with seed: " ['my', 'word,', 'the', "father's", 'son:', "i'll", "swear,'tis", 'a', 'very', 'pretty', 'boy.', "o'", 'my', 'troth,', 'i', 'looked', 'upon', 'him', "o'", 'wednesday', 'half', 'an', 'hour', 'together:', 'has', 'such', 'a', 'confirmed', 'countenance.', 'i'] "


my word, the father's son: i'll swear,'tis a very pretty boy. o' my troth, i looked upon him o' wednesday half an hour together: has such a confirmed countenance. i

 the i and i and the the the you and the a the the and i and the to of of the to of you the coriolanus: and you of my a you and the 

# GPT custom

In [ ]:
import csv

with open('/content/leotol.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('/content/leotol.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

In [ ]:
!pip install -q aitextgen #install the main package

     |████████████████████████████████| 572 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 30.5 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 919 kB 27.8 MB/s 
     |████████████████████████████████| 636 kB 45.2 MB/s 
     |████████████████████████████████| 272 kB 50.2 MB/s 
     |████████████████████████████████| 119 kB 51.1 MB/s 
     |████████████████████████████████| 829 kB 44.3 MB/s 
     |████████████████████████████████| 1.3 MB 35.1 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
     |████████████████████████████████| 294 kB 47.4 MB/s 
     |████████████████████████████████| 142 kB 42.7 MB/s 


In [ ]:
from aitextgen import aitextgen

In [ ]:
! nvidia-smi

Wed Sep  1 14:01:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)


Fetching checkpoint:   0%|          | 0.00/77.0 [00:00<?, ?it/s]

Fetching hparams.json:   0%|          | 0.00/90.0 [00:00<?, ?it/s]

Fetching model.ckpt.data-00000-of-00001:   0%|          | 0.00/498M [00:00<?, ?it/s]

Fetching model.ckpt.index:   0%|          | 0.00/5.21k [00:00<?, ?it/s]

Fetching model.ckpt.meta:   0%|          | 0.00/471k [00:00<?, ?it/s]

Converting TensorFlow checkpoint from /content/aitextgen/124M
Loading TF weight model/h0/attn/c_attn/b with shape [2304]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_proj/b with shape [768]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 768, 768]
Loading TF weight model/h0/ln_1/b with shape [768]
Loading TF weight model/h0/ln_1/g with shape [768]
Loading TF weight model/h0/ln_2/b with shape [768]
Loading TF weight model/h0/ln_2/g with shape [768]
Loading TF weight model/h0/mlp/c_fc/b with shape [3072]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 768, 3072]
Loading TF weight model/h0/mlp/c_proj/b with shape [768]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 3072, 768]
Loading TF weight model/h1/attn/c_attn/b with shape [2304]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h1/attn/c_proj/b with shape [768]
Loading TF weight model/h1/attn/c_proj/w with shape [1, 7

Save PyTorch model to aitextgen/pytorch_model.bin
Save configuration file to aitextgen/config.json


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
df = pd.read_fwf('/content/leotol.txt')
df.to_csv('/content/leotol.csv')

input_file = pd.read_csv("/content/leotol.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
ai.train('/content/shakespeare_train.txt',
         line_by_line=False,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1)

  0%|          | 0/10001 [00:00<?, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/500 [00:00<?, ?it/s]

100 steps reached: generating sample texts.

Messenger.

Messenger:
And to the west,
In the west, and in the middle,
I go forth the common report.

Messenger:
Here, my lord.

Messenger:
If God keep him from all this,
We are going to the throne.

Messenger:

My grace, my lord, we will be consul.

Messenger:
And, you know your reasons,
When you have letters from the king,
Your reasons must be the enemy's son, and his friends,
To give you reasons why you have been consul.

Messenger:
Good morrow, good morrow, my lord, the king's son!

Messenger:
God give me some reasons with what you have heard,
You have noted the circumstances of the other.

Messenger:
God save your cause and with some sort of the other,
To the end.

COMINIUS:
Well then, as I am,
You have been a witness to me,
Your own reasons and their reasons,
Were you your own eyes,
You have been a witness to the war,

200 steps reached: generating sample texts.
.

COMINIUS:
My troth is not your house?
The gods forbid
I cannot pass: I

#Generate (added)

In [ ]:
ai = aitextgen(model_folder="/content/trained_model", to_gpu=True)

In [ ]:
ai.generate(n=3,
            batch_size=1,
            max_length=50,
            temperature=1.0,
            top_p=0.9)

that honour wherein he stands.

HASTINGS:
I'll be sworn by, that he shall never reign.

HASTINGS:
So stood the state so?

HASTINGS:
How! I would
'st: remains!
You have received many wounds for your country.

SICINIUS:
That will not remain
The way to his house; and many are
begged.

SICINIUS
KING EDWARD IV:
Then speak at once what is it thou demand'st
Of my imprisonment.

DUCHESS OF YORK:
I am, Lord Hastings; I will shortly send thy soul to heaven.
